Lambda School Data Science

*Unit 2, Sprint 1, Module 3*

---

# Ridge Regression

## Assignment

We're going back to our other **New York City** real estate dataset. Instead of predicting apartment rents, you'll predict property sales prices.

But not just for condos in Tribeca...

Instead, predict property sales prices for **One Family Dwellings** (`BUILDING_CLASS_CATEGORY` == `'01 ONE FAMILY DWELLINGS'`). 

Use a subset of the data where the **sale price was more than \\$100 thousand and less than $2 million.** 

The [NYC Department of Finance](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page) has a glossary of property sales terms and NYC Building Class Code Descriptions. The data comes from the [NYC OpenData](https://data.cityofnewyork.us/browse?q=NYC%20calendar%20sales) portal.

- [X] Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.
- [X] Do one-hot encoding of categorical features.
- [X] Do feature selection with `SelectKBest`.
- [X] Do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html).
- [X] Fit a ridge regression model with multiple features.
- [X] Get mean absolute error for the test set.
- [X] As always, commit your notebook to your fork of the GitHub repo.


## Stretch Goals
- [ ] Add your own stretch goal(s) !
- [ ] Instead of `Ridge`, try `LinearRegression`. Depending on how many features you select, your errors will probably blow up! 💥
- [ ] Instead of `Ridge`, try [`RidgeCV`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html).
- [ ] Learn more about feature selection:
    - ["Permutation importance"](https://www.kaggle.com/dansbecker/permutation-importance)
    - [scikit-learn's User Guide for Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html)
    - [mlxtend](http://rasbt.github.io/mlxtend/) library
    - scikit-learn-contrib libraries: [boruta_py](https://github.com/scikit-learn-contrib/boruta_py) & [stability-selection](https://github.com/scikit-learn-contrib/stability-selection)
    - [_Feature Engineering and Selection_](http://www.feat.engineering/) by Kuhn & Johnson.
- [ ] Try [statsmodels](https://www.statsmodels.org/stable/index.html) if you’re interested in more inferential statistical approach to linear regression and feature selection, looking at p values and 95% confidence intervals for the coefficients.
- [ ] Read [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf), Chapters 1-3, for more math & theory, but in an accessible, readable way.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

In [22]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [23]:
import pandas as pd
import numpy as np
import pandas_profiling

# Read New York City property sales data
df = pd.read_csv(DATA_PATH+'condos/NYC_Citywide_Rolling_Calendar_Sales.csv',
                 thousands=',', engine='python')

# Change column names: replace spaces with underscores
df.columns = [col.replace(' ', '_') for col in df]

# SALE_PRICE was read as strings.
# Remove symbols, convert to integer
df['SALE_PRICE'] = (
    df['SALE_PRICE']
    .str.replace('$','')
    .str.replace('-','')
    .str.replace(',','')
    .astype(int)
)

In [24]:
# BOROUGH is a numeric column, but arguably should be a categorical feature,
# so convert it from a number to a string
df['BOROUGH'] = df['BOROUGH'].astype(str)

In [25]:
# Reduce cardinality for NEIGHBORHOOD feature

# Get a list of the top 10 neighborhoods
top10 = df['NEIGHBORHOOD'].value_counts()[:10].index

# At locations where the neighborhood is NOT in the top 10, 
# replace the neighborhood with 'OTHER'
df.loc[~df['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'

In [26]:
# Filter the data for One Family Dwellings
df = df[df['BUILDING_CLASS_CATEGORY'] == '01 ONE FAMILY DWELLINGS']

# Filter the sale price between $100K and $2 million
df = df[(df['SALE_PRICE'] >= 100000) & (df['SALE_PRICE'] <= 2000000)]

# Drop the EASE-MENT and APARTMENT_NUMBER columns (full of NaNs)
df = df.drop(columns=['EASE-MENT', 'APARTMENT_NUMBER'])

# Convert LAND_SQUARE_FEET to numerical
df['LAND_SQUARE_FEET'] = pd.to_numeric(df['LAND_SQUARE_FEET'])

In [27]:
df.head()

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,BUILDING_CLASS_AT_PRESENT,ADDRESS,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
44,3,OTHER,01 ONE FAMILY DWELLINGS,1,5495,801,A9,4832 BAY PARKWAY,11230.0,1.0,0.0,1.0,6800,1325.0,1930.0,1,A9,550000,01/01/2019
61,4,OTHER,01 ONE FAMILY DWELLINGS,1,7918,72,A1,80-23 232ND STREET,11427.0,1.0,0.0,1.0,4000,2001.0,1940.0,1,A1,200000,01/01/2019
78,2,OTHER,01 ONE FAMILY DWELLINGS,1,4210,19,A1,1260 RHINELANDER AVE,10461.0,1.0,0.0,1.0,3500,2043.0,1925.0,1,A1,810000,01/02/2019
108,3,OTHER,01 ONE FAMILY DWELLINGS,1,5212,69,A1,469 E 25TH ST,11226.0,1.0,0.0,1.0,4000,2680.0,1899.0,1,A1,125000,01/02/2019
111,3,OTHER,01 ONE FAMILY DWELLINGS,1,7930,121,A5,5521 WHITTY LANE,11203.0,1.0,0.0,1.0,1710,1872.0,1940.0,1,A5,620000,01/02/2019


In [28]:
df.shape

(3164, 19)

### Do train-test split. Use data from January — March 2019 to train. Use data from April 2019 to test.

In [29]:
df['SALE_DATE'] = pd.to_datetime(df['SALE_DATE'], infer_datetime_format=True)
df['SALE_DATE'].dt.month.value_counts()

1    952
3    802
2    763
4    647
Name: SALE_DATE, dtype: int64

In [34]:
train = df[df['SALE_DATE'].dt.month < 4]
test  = df[df['SALE_DATE'].dt.month == 4]
train.shape, test.shape

((2517, 19), (647, 19))

In [36]:
# Replace "0" values in GROSS/LAND_SQUARE_FEET to mean
gross_sq_ft = train['GROSS_SQUARE_FEET'].copy()
condition = (gross_sq_ft == 0)
train.loc[condition, 'GROSS_SQUARE_FEET'] = gross_sq_ft.mean()

land_sq_ft = train['LAND_SQUARE_FEET'].copy()
condition = (land_sq_ft == 0)
train.loc[condition, 'LAND_SQUARE_FEET'] = land_sq_ft.mean()

In [37]:
train.profile_report()

In [38]:
train.select_dtypes(exclude='number').describe().T.sort_values(by='unique')

,count,unique,top,freq,first,last
BUILDING_CLASS_CATEGORY,2517,1,01 ONE FAMILY DWELLINGS,2517,NaN,NaN
TAX_CLASS_AT_PRESENT,2517,2,1,2486,NaN,NaN
BOROUGH,2517,5,4,1209,NaN,NaN
NEIGHBORHOOD,2517,6,OTHER,2390,NaN,NaN
BUILDING_CLASS_AT_TIME_OF_SALE,2517,11,A1,921,NaN,NaN
BUILDING_CLASS_AT_PRESENT,2517,13,A1,921,NaN,NaN
SALE_DATE,2517,68,2019-01-31 00:00:00,78,2019-01-01 00:00:00,2019-03-30 00:00:00
ADDRESS,2517,2507,117-45 125TH STREET,2,NaN,NaN


In [39]:
# Drop the non-numeric columns with high cardinality and target column
target = 'SALE_PRICE'
max_cardinality = 50  # arbitrary cut-off

high_cardinality = [col for col in train.select_dtypes(exclude=np.number)
                    if train[col].nunique() > max_cardinality]

# Also drop the constant features in the DataFrame
to_drop = ['BUILDING_CLASS_CATEGORY', 'TAX_CLASS_AT_TIME_OF_SALE']

features = train.columns.drop([target] + to_drop + high_cardinality)

X_train = train[features]
X_test = test[features]
y_train = train[target]
y_test = test[target]

### Do one-hot encoding of categorical features.

In [40]:
import category_encoders as ce

In [41]:
encoder = ce.OneHotEncoder(use_cat_names=True)
X_train = encoder.fit_transform(X_train)
X_test = encoder.transform(X_test)

In [42]:
X_train.shape

(2517, 46)

### Do feature selection with `SelectKBest`.

In [43]:
from sklearn.feature_selection import f_regression, SelectKBest
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

In [44]:
for k in range(1, len(X_train.columns)+1):
    if k==1:
        print('1 feature')
    else:
        print(f'{k} features')
    
    selector = SelectKBest(score_func=f_regression, k=k)
    X_train_selected = selector.fit_transform(X_train, y_train)
    X_test_selected = selector.transform(X_test)
    
    model = LinearRegression()
    model.fit(X_train_selected, y_train)
    
    y_pred = model.predict(X_test_selected)
    mae = mean_absolute_error(y_test, y_pred)
    print(f'Test MAE: ${mae:,.0f} \n')

1 feature
Test MAE: $185,550 

2 features
Test MAE: $181,608 

3 features
Test MAE: $181,294 

4 features
Test MAE: $181,294 

5 features
Test MAE: $172,891 

6 features
Test MAE: $172,934 

7 features
Test MAE: $170,495 

8 features
Test MAE: $170,595 

9 features
Test MAE: $170,290 

10 features
Test MAE: $167,694 

11 features
Test MAE: $158,763 

12 features
Test MAE: $159,516 

13 features
Test MAE: $160,339 

14 features
Test MAE: $160,041 

15 features
Test MAE: $160,103 

16 features
Test MAE: $160,104 

17 features
Test MAE: $159,909 

18 features
Test MAE: $159,909 

19 features
Test MAE: $159,735 

20 features
Test MAE: $159,735 

21 features
Test MAE: $159,812 

22 features
Test MAE: $159,812 

23 features
Test MAE: $157,581 

24 features
Test MAE: $157,581 

25 features
Test MAE: $157,581 

26 features
Test MAE: $157,581 

27 features
Test MAE: $157,495 

28 features
Test MAE: $157,456 

29 features
Test MAE: $157,435 

30 features
Test MAE: $157,421 

31 features
Test MAE

### Do feature scaling. 

In [45]:
from sklearn.preprocessing import StandardScaler

In [46]:
# Scale data before doing ridge regression
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

### Fit a ridge regression model with multiple features.

In [48]:
from sklearn.linear_model import Ridge

In [49]:
model = Ridge()
model.fit(X_train_scaled, y_train)

Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)

### Get mean absolute error for the test set.

In [50]:
# Get Test MAE
y_pred = model.predict(X_test_scaled)
mae = mean_absolute_error(y_test, y_pred)
print(f'Test Mean Absolute Error: ${mae:,.0f}')

Test Mean Absolute Error: $159,665


In [52]:
from sklearn.metrics import mean_squared_error

In [63]:
alphas = []
mses = []

for alpha in np.logspace(-6, 6, num=13):
    ridge_reg_split = Ridge(alpha=alpha).fit(X_train, y_train)
    mse = mean_squared_error(y_test, ridge_reg_split.predict(X_test))
    print(alpha, mse)
    alphas.append(alpha)
    mses.append(mse)

1e-06 53797989169.896904
1e-05 53797986845.03989
0.0001 53797963428.168205
0.001 53797730012.59393
0.01 53795470200.05253
0.1 53778673163.90552
1.0 53755177362.69993
10.0 54088004363.391624
100.0 55303512129.30392
1000.0 56617287970.002655
10000.0 57554955250.90829
100000.0 57668688643.88111
1000000.0 57383343483.59357
